# Query PDF Tutorial

In this tutorial, we demonstrate how to load a PDF and query it.

<table align="left">
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/Yongqi099/evadb/blob/staging/apps/youtube_summary/summary.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" /> Run on Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/georgia-tech-db/eva/blob/master/tutorials/12-query-pdf.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" /> View source on GitHub</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/georgia-tech-db/eva/raw/master/tutorials/12-query-pdf.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png" /> Download notebook</a>
  </td>
</table><br><br>

### Connect to EvaDB


In [1]:
%pip install --quiet "evadb[document,notebook]"
%pip install youtube_transcript_api
%pip install reportlab
%pip install gpt4all

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 530.1/530.1 kB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.6/137.6 kB 17.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.9/108.9 kB 14.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 kB 10.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 75.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 70.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 76.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.0/77.0 kB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 95.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 80.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 11.7 MB/s eta 0:0

Downloading: "http://ml.cs.tsinghua.edu.cn/~chenxi/pytorch-models/mnist-b07bb66b.pth" to /root/.cache/torch/hub/checkpoints/mnist-b07bb66b.pth
100%|██████████| 1.03M/1.03M [00:01<00:00, 803kB/s] 
Downloading: "https://download.pytorch.org/models/fasterrcnn_resnet50_fpn_coco-258fb6c6.pth" to /root/.cache/torch/hub/checkpoints/fasterrcnn_resnet50_fpn_coco-258fb6c6.pth


In [ ]:
import evadb
cursor = evadb.connect().cursor()
import warnings
warnings.filterwarnings("ignore")

from google.colab import output
output.enable_custom_widget_manager()

from youtube_transcript_api import YouTubeTranscriptApi

from reportlab.platypus import SimpleDocTemplate, Paragraph
from reportlab.lib.styles import getSampleStyleSheet

from gpt4all import GPT4All

In [37]:
gpt = GPT4All("ggml-model-gpt4all-falcon-q4_0.bin")

Found model file at  /root/.cache/gpt4all/ggml-model-gpt4all-falcon-q4_0.bin


### Video Link

In [13]:
# replace with your video URL
video_url = 'https://www.youtube.com/watch?v=dQw4w9WgXcQ&pp=ygUNbmV2ZXIgZ2l2ZSB1cA%3D%3D'

### Get Youtube Transcript

In [14]:
# Check if the video URL starts with 'https://www.youtube.com/watch'
if not video_url.startswith('https://www.youtube.com/watch'):
    raise ValueError("Invalid video URL. It should start with 'https://www.youtube.com/watch'")

# Extract the video ID from the URL
video_id = video_url.split('=')[1]

# Get the transcript of the video
transcript = YouTubeTranscriptApi.get_transcript(video_id)

In [20]:
# Combine all the parts into a single string
full_transcript = " ".join([part['text'] for part in transcript])
# Write the transcript to a text file
with open('transcript.txt', 'w') as f:
    f.write(full_transcript)

# Write the transcript to a PDF file
doc = SimpleDocTemplate("pdf_sample1.pdf")
styles = getSampleStyleSheet()
story = [Paragraph(full_transcript, styles["BodyText"])]
doc.build(story)

In [41]:
# Prompt the model to summarize the transcript
prompt = "Summarize the following text: " + full_transcript
summary = gpt.generate(prompt)


In [43]:
print(len(full_transcript))
print(len(summary))

1848
896


### Hugging Face Summary

Load PDFs

In [25]:
cursor.query("DROP TABLE IF EXISTS MyPDFs").df()
cursor.query("LOAD PDF 'pdf_sample1.pdf' INTO MyPDFs").df()

,0
0,Number of loaded PDF: 1


Retrieve Text from Loaded PDFs

In [26]:
cursor.query("SELECT * FROM MyPDFs").df()

,mypdfs._row_id,mypdfs.name,mypdfs.page,mypdfs.paragraph,mypdfs.data
0,1,pdf_sample1.pdf,1,1,[Music] we're no strangers to love you know th...


In [27]:
cursor.query("""
    SELECT *
    FROM MyPDFs
""").df()

,mypdfs._row_id,mypdfs.name,mypdfs.page,mypdfs.paragraph,mypdfs.data
0,1,pdf_sample1.pdf,1,1,[Music] we're no strangers to love you know th...


Create UDFs for Text Classification and Text Summarization

In [28]:
cursor.query("""
    CREATE FUNCTION IF NOT EXISTS TextClassifier
    TYPE HuggingFace
    TASK 'text-classification'
    MODEL 'distilbert-base-uncased-finetuned-sst-2-english'
""").df()

,0
0,"Function TextClassifier already exists, nothin..."


In [29]:
cursor.query("""
    CREATE FUNCTION IF NOT EXISTS TextSummarizer
    TYPE HuggingFace
    TASK 'summarization'
    MODEL 'facebook/bart-large-cnn'
""").df()

,0
0,"Function TextSummarizer already exists, nothin..."


Get Summaries of a Subset of Paragraphs with Negative Sentiment

In [30]:
cursor.query("""
    SELECT data, TextSummarizer(data)
    FROM MyPDFs
""").df()


,mypdfs.data,textsummarizer.summary_text
0,[Music] we're no strangers to love you know th...,"""We've known each other for so long your heart..."
